In [1]:
import psycopg2


In [ ]:
conn = None
    # try:
    # connect to the PostgreSQL server
    print('Connecting to the PostgreSQL database...')
    conn = psycopg2.connect(
        database="smartrecovery", user="admin", password="secret", host="postgres", port=5432)

    # create a cursor
    cur = conn.cursor()
    cur.execute(
        r'ALTER TABLE dimensionZona ADD PRIMARY KEY (LlavePuntoConsumo);')
    cur.close()

In [10]:
df = (
    spark.read
    .format("jdbc")
    .option("url", "jdbc:postgresql://postgres/smartrecovery")
    .option("dbtable", "public.\"dimensionZona\"")
    .option("user", "admin")
    .option("password", "secret")
    .load()
)
df.show()

+---------+----------+--------------+------------+-----------------+------------+--------------+-------------+-------------+-------------------+-------------------+----------------+--------------+------------------+----+-------+-------------+----------------+-----------+
|LlaveZona|CodigoZona|    NombreZona|CodigoRegion|     NombreRegion|CodigoCentro|  NombreCentro|CodigoOficina|NombreOficina|NombreCentroTecnico|FechaInicioRegistro|FechaFinRegistro|EstadoRegistro|VisibleFacturacion|Ruta|Empresa|CodigoEmpresa|CodigoItinerario| Itinerario|
+---------+----------+--------------+------------+-----------------+------------+--------------+-------------+-------------+-------------------+-------------------+----------------+--------------+------------------+----+-------+-------------+----------------+-----------+
|        1|         0|Sin definición|        null|   Sin definición|        null|Sin definición|         null|         null|               null|         2023-03-30|            null|   

In [1]:
# importaremos las librerias necesarias
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import Row
from pyspark.sql import functions as F
from pyspark.ml import Pipeline
from pyspark import SparkConf
import os
import shutil
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import concat_ws, first,col, radians, asin, sin, sqrt, cos, max, min, lit,to_timestamp,to_date, when, isnan, when, count, col
from pyspark.sql import Window as W
from dotenv import load_dotenv

In [2]:
spark = SparkSession\
    .builder.master(("local[*]"))\
    .config("spark.driver.memory", "15g")\
    .config("spark.driver.extraClassPath", "postgresql-42.5.3.jar:aws-java-sdk-bundle-1.12.402.jar:aws-java-sdk-core-1.12.402.jar:aws-java-sdk-s3-1.12.402.jar:hadoop-aws-3.3.2.jar")\
    .config("spark.hadoop.fs.s3a.endpoint", 'http://minio:9000')\
    .config("spark.hadoop.fs.s3a.access.key", 'minio')\
    .config("spark.hadoop.fs.s3a.secret.key", 'minio123')\
    .config("spark.hadoop.fs.s3a.path.style.access", True)\
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")\
    .config("spark.history.fs.logDirectory","s3a://spark/spark_logs")\
    .config("spark.eventLog.enabled","true")\
    .config("spark.eventLog.dir","s3a://spark/spark_logs")\
    .config("spark.driver.extraJavaOptions","-Xss4M")\
    .getOrCreate()

In [ ]:
DXHS_FACTURACION_CONSUMOS = spark.read.option("header", "true").parquet(f"s3a://data-lake/DWH_datadwh/DXES_SERVICIOS.parquet")


In [4]:
DXHS_FACTURACION_CONSUMOS.show()

+-----------+-----------+-----------+----------+---------+------------+-----------+-----------+---------+---------------+-----------+----------+-------+----------------+----------+----------------+----------------------+----------------------+----------------------+---------------+-----------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------------+--------------------+---------+---------+---------+----------+-----------+--------+------------------+-----------+--------+---------+-------------+-----------------+---------------+--------------------+--------------------+--------+------------+-----------------+---------------+---------------------+----------------+---------+-------------+--------------------+--------------------+--------------------+---------+---------+---------+--------------------+------+-------------------+-------------------+-------------------+-------------------+---------+-----------------+--------

# CONSUMOSNORMFACT parquet

## LECTURA DXHS_FACTURACION_CONSUMOS

In [3]:
sourceBucket = "data-lake"
data = ['DWH_datadwh/dxhs_facturacion_consumos_truncate.parquet','DWH_datadwh/dxhs_servicios_truncate.parquet'\
        ,'DWH_datadwh/DXAA_GEO_BARRIOS.parquet','DWH_datadwh/dxhs_lecturas_truncate.parquet']

In [4]:
DXHS_FACTURACION_CONSUMOS = spark.read.option("header", "true").parquet(f"s3a://{sourceBucket}/{data[0]}")\
.select('ID_SERVICIO', 'ID_MES', 'CS_FACTURADO', 'CS_ANULADO',
        'CS_REFACTURADO', 'IN_FACTURADO', 'CC_DIAS_FACTURADO', 'FE_FACTURA_ULTIMA').alias("DXHS_FACTURACION_CONSUMOS")


In [69]:
outputPath = f"s3a://{sourceBucket}/DWH_datadwh/dxhs_facturacion_consumos_truncate.parquet"

In [5]:
DXHS_FACTURACION_CONSUMOS.show()

+-----------+------+------------+----------+--------------+------------+-----------------+-------------------+
|ID_SERVICIO|ID_MES|CS_FACTURADO|CS_ANULADO|CS_REFACTURADO|IN_FACTURADO|CC_DIAS_FACTURADO|  FE_FACTURA_ULTIMA|
+-----------+------+------------+----------+--------------+------------+-----------------+-------------------+
|    3139055|202102|           0|         0|             0|           S|               31|2021-02-19 00:00:00|
|    2500103|202102|          78|         0|             0|           S|               32|2021-02-16 00:00:00|
|    2479906|202102|         277|         0|             0|           S|               32|2021-02-10 00:00:00|
|    3024931|202102|         241|         0|             0|           S|               32|2021-02-10 00:00:00|
|    2506529|202102|          43|         0|             0|           S|               31|2021-02-19 00:00:00|
|    2503351|202102|          92|         0|             0|           S|               32|2021-02-23 00:00:00|
|

In [77]:
update= DXHS_FACTURACION_CONSUMOS.filter(col("ID_MES")>=202301)#orderBy(col("ID_MES").desc()).show()
print(update.count())
update.orderBy(col("ID_MES").desc()).show()

0
+-----------+------+------------+----------+--------------+------------+-----------------+-----------------+
|ID_SERVICIO|ID_MES|CS_FACTURADO|CS_ANULADO|CS_REFACTURADO|IN_FACTURADO|CC_DIAS_FACTURADO|FE_FACTURA_ULTIMA|
+-----------+------+------------+----------+--------------+------------+-----------------+-----------------+
+-----------+------+------------+----------+--------------+------------+-----------------+-----------------+



In [73]:
update.write.mode("overwrite").parquet(outputPath)

In [69]:
DXHS_FACTURACION_CONSUMOS.filter((col("IN_FACTURADO")=="N")&(col("ID_MES")=="202301")).count()

71275

In [71]:
DXHS_FACTURACION_CONSUMOS.filter((col("IN_FACTURADO")=="S")&(col("ID_MES")=="202302")).count()

1804025

## LECTURA QUERY_DXHS_SERVICIOS

In [5]:
DXHS_SERVICIOS = spark.read.option("header", "true").parquet(f"s3a://{sourceBucket}/{data[1]}")\
.select('ID_SERVICIO','ID_CLIENTE','TI_SERVICIO', 'TI_TARIFA','TI_TENSION','TI_TARIFA_GRUPO', 'ID_MES', 'ID_BARRIO').alias("DXHS_SERVICIOS")



In [6]:
DXHS_SERVICIOS.show()

+-----------+----------+-----------+---------+----------+---------------+------+---------+
|ID_SERVICIO|ID_CLIENTE|TI_SERVICIO|TI_TARIFA|TI_TENSION|TI_TARIFA_GRUPO|ID_MES|ID_BARRIO|
+-----------+----------+-----------+---------+----------+---------------+------+---------+
|    5569230|  50582280|      SU011|      B10|     TT001|              1|201711|     5877|
|    5499047|  50508215|      SU011|      B20|     TT001|              1|201711|     2007|
|    5687079|  50420879|      SU011|      B20|     TT001|              1|201711|     7431|
|    5524754|  50371905|      SU011|      B20|     TT001|              2|201711|      292|
|    5630440|  50158968|      SU011|      B20|     TT001|              1|201711|     1633|
|    5533487|  50837829|      SU011|      B20|     TT001|              1|201711|      890|
|    5607192|  50621558|      SU011|      B10|     TT001|              1|201711|     5313|
|    5574872|  50586906|      SU011|      B10|     TT001|              1|201711|     4946|

In [43]:
DXHS_SERVICIOS.count()

180864663

In [26]:
update = DXHS_SERVICIOS.filter(col("ID_MES")<202301)
update.show()
print(update.count())

+-----------+----------+-----------+---------+----------+---------------+------+---------+
|ID_SERVICIO|ID_CLIENTE|TI_SERVICIO|TI_TARIFA|TI_TENSION|TI_TARIFA_GRUPO|ID_MES|ID_BARRIO|
+-----------+----------+-----------+---------+----------+---------------+------+---------+
|    2125615|  20125610|      SU011|      B20|     TT001|              1|201701|     1044|
|    2019814|  20019813|      SU011|      B20|     TT001|              2|201701|     1651|
|    2166622|  20166616|      SU011|      B20|     TT001|              1|201701|      857|
|    2153792|  20153787|      SU011|      B20|     TT001|              1|201701|    11693|
|    2040790|  20040789|      SU011|      B20|     TT001|              1|201701|     1981|
|    2038733|  20038732|      SU011|      B20|     TT001|              1|201701|     1977|
|    2155168|  20155163|      SU011|      B20|     TT001|              1|201701|      793|
|    1028492|  10028492|      SU011|      B20|     TT001|              1|201701|       96|

In [63]:
DXHS_LECTURAS.write.mode("overwrite").parquet(outputPath)

In [3]:
spark.read.option("header", "true").parquet(f"s3a://data-lake/preprocessed_tables/consumosNormFact.parquet").count()


141413952

In [3]:
spark.read.option("header", "true").parquet(f"s3a://data-lake/preprocessed_tables/calculosNorm.parquet").count()


217371795

In [6]:
spark.read.option("header", "true").parquet(f"s3a://data-lake/preprocessed_tables/calculosProm.parquet").count()

123082007

# LECTURA DXAA_GEO_BARRIOS

In [6]:
DXAA_GEO_BARRIOS = spark.read.option("header", "true").parquet(f"s3a://{sourceBucket}/{data[2]}")\
.alias("GEO_BARRIOS")

DXAA_GEO_BARRIOS.show()

+---------+---------+------------+---------------+---------+-------------------+
|ID_BARRIO|DE_BARRIO|ID_MUNICIPIO|ID_DEPARTAMENTO|ID_ESTADO|           FH_CARGA|
+---------+---------+------------+---------------+---------+-------------------+
|     4777|   ZONA 0|        4048|             17|        2|2023-02-10 21:00:25|
|     4778|   ZONA 0|        4049|            290|        8|2023-02-10 21:00:25|
|     4779|   ZONA 0|        4050|            290|        8|2023-02-10 21:00:25|
|     4780|   ZONA 0|        4051|            290|        8|2023-02-10 21:00:25|
|     4781|   ZONA 0|        4052|            290|        8|2023-02-10 21:00:25|
|     4782|   ZONA 0|        4053|            290|        8|2023-02-10 21:00:25|
|     4783|   ZONA 0|        4054|            290|        8|2023-02-10 21:00:25|
|     4784|   ZONA 0|        4055|            290|        8|2023-02-10 21:00:25|
|     4785|   ZONA 0|        4056|             21|        2|2023-02-10 21:00:25|
|     4786|   ZONA 0|       

In [45]:
DXAA_GEO_BARRIOS.count()

12158

# LECTURA dxhs_lecturas

In [7]:
DXHS_LECTURAS = spark.read.option("header", "true").parquet(f"s3a://{sourceBucket}/{data[3]}")\
.select("ID_APARATO","ID_MES","ID_SERVICIO").alias("DXHS_LECTURAS")

DXHS_LECTURAS.show()

+----------+------+-----------+
|ID_APARATO|ID_MES|ID_SERVICIO|
+----------+------+-----------+
|008D804259|201802|    5690527|
|014H965841|201802|    6203585|
|011D809340|201802|    3213243|
|  10656411|201802|    5065896|
|012D828882|201803|    5952013|
|012D806314|201802|    3201599|
|0098879261|201803|    1005729|
|016Q868685|201803|    5558487|
|0990055834|201802|    2371428|
|011D851852|201802|    5299792|
|  00529067|201802|    5044153|
|006C004561|201802|    5066165|
| 020320106|201803|    5364057|
| 990047156|201803|    5091418|
| 000578835|201803|    5034519|
|014D843749|201803|    5791962|
|005B146016|201803|    3127152|
|013D934035|201803|    3241069|
|005C168954|201803|    5592654|
| 990039239|201803|    5178756|
+----------+------+-----------+
only showing top 20 rows



In [67]:
update = DXHS_LECTURAS.filter(col("ID_MES")<=202301)
update.show()

+----------+------+-----------+
|ID_APARATO|ID_MES|ID_SERVICIO|
+----------+------+-----------+
|008D804259|201802|    5690527|
|014H965841|201802|    6203585|
|011D809340|201802|    3213243|
|  10656411|201802|    5065896|
|012D828882|201803|    5952013|
|012D806314|201802|    3201599|
|0098879261|201803|    1005729|
|016Q868685|201803|    5558487|
|0990055834|201802|    2371428|
|011D851852|201802|    5299792|
|  00529067|201802|    5044153|
|006C004561|201802|    5066165|
| 020320106|201803|    5364057|
| 990047156|201803|    5091418|
| 000578835|201803|    5034519|
|014D843749|201803|    5791962|
|005B146016|201803|    3127152|
|013D934035|201803|    3241069|
|005C168954|201803|    5592654|
| 990039239|201803|    5178756|
+----------+------+-----------+
only showing top 20 rows



JOINS

In [8]:
join_1  =DXHS_FACTURACION_CONSUMOS.join(DXHS_SERVICIOS, (col('DXHS_FACTURACION_CONSUMOS.ID_SERVICIO')==col('DXHS_SERVICIOS.ID_SERVICIO')) \
                                        & (col('DXHS_FACTURACION_CONSUMOS.ID_MES')==col('DXHS_SERVICIOS.ID_MES')),'inner') \
                                        .drop(col('DXHS_SERVICIOS.ID_MES')).drop(col('DXHS_SERVICIOS.ID_SERVICIO')).cache()
 
join_1.show()

+-----------+------+------------+----------+--------------+------------+-----------------+-------------------+----------+-----------+---------+----------+---------------+---------+
|ID_SERVICIO|ID_MES|CS_FACTURADO|CS_ANULADO|CS_REFACTURADO|IN_FACTURADO|CC_DIAS_FACTURADO|  FE_FACTURA_ULTIMA|ID_CLIENTE|TI_SERVICIO|TI_TARIFA|TI_TENSION|TI_TARIFA_GRUPO|ID_BARRIO|
+-----------+------+------------+----------+--------------+------------+-----------------+-------------------+----------+-----------+---------+----------+---------------+---------+
|    1000006|202106|           0|         0|             0|           S|               28|2021-06-02 00:00:00|  10000006|      SU011|      B20|     TT001|              1|       15|
|    1000014|202105|          26|         0|             0|           S|               30|2021-05-05 00:00:00|  10006316|      SU011|      B20|     TT001|              1|       15|
|    1000019|201706|          61|         0|             0|           S|               32|2017-

In [ ]:
#  'ID_MES','ID_SERVICIO','TI_TARIFA','TI_TARIFA_GRUPO','FE_FACTURA_ULTIMO' is unique but it is treated in the sql script as not??
col_list = ['CS_FACTURADO', 'CS_ANULADO', 'CS_REFACTURADO']
x = join_1.groupBy('ID_MES','ID_SERVICIO','TI_TARIFA','TI_TARIFA_GRUPO','FE_FACTURA_ULTIMA').count().orderBy(col('count').desc())
x.show()



In [9]:
col_list = ['CS_FACTURADO', 'CS_ANULADO', 'CS_REFACTURADO']
csm_sum = join_1.withColumn('CSMO',sum([F.col(c) for c in col_list])
).withColumn('IND_FACT', when(col('IN_FACTURADO') == 'S','Si').otherwise('No'))

csm_sum = csm_sum.withColumn('DIAS_FACT', when(col('CC_DIAS_FACTURADO').isNull(),0).otherwise(col('CC_DIAS_FACTURADO')))
csm_sum = csm_sum.withColumn('CSMO_D',when(col('DIAS_FACT')!=0,\
                                           sum([F.col(c) for c in col_list])/col('CC_DIAS_FACTURADO')).otherwise(0))\
.withColumn("FE_FACT", to_date(col('FE_FACTURA_ULTIMA'),"YYYY-MM-DD"))


csm_sum.show()

#TODO CSMO_D y FE_FACT

+-----------+------+------------+----------+--------------+------------+-----------------+-------------------+----------+-----------+---------+----------+---------------+---------+----+--------+---------+---------+----------+
|ID_SERVICIO|ID_MES|CS_FACTURADO|CS_ANULADO|CS_REFACTURADO|IN_FACTURADO|CC_DIAS_FACTURADO|  FE_FACTURA_ULTIMA|ID_CLIENTE|TI_SERVICIO|TI_TARIFA|TI_TENSION|TI_TARIFA_GRUPO|ID_BARRIO|CSMO|IND_FACT|DIAS_FACT|   CSMO_D|   FE_FACT|
+-----------+------+------------+----------+--------------+------------+-----------------+-------------------+----------+-----------+---------+----------+---------------+---------+----+--------+---------+---------+----------+
|    1000006|202106|           0|         0|             0|           S|               28|2021-06-02 00:00:00|  10000006|      SU011|      B20|     TT001|              1|       15|   0|      Si|       28| 0.000000|2021-06-02|
|    1000014|202105|          26|         0|             0|           S|               30|2021-0

In [13]:
csm_sum.filter(col('ID_SERVICIO')==1000006).orderBy('ID_MES').show()

+-----------+------+------------+----------+--------------+------------+-----------------+-------------------+----------+-----------+---------+----------+---------------+---------+----+--------+---------+--------+----------+
|ID_SERVICIO|ID_MES|CS_FACTURADO|CS_ANULADO|CS_REFACTURADO|IN_FACTURADO|CC_DIAS_FACTURADO|  FE_FACTURA_ULTIMA|ID_CLIENTE|TI_SERVICIO|TI_TARIFA|TI_TENSION|TI_TARIFA_GRUPO|ID_BARRIO|CSMO|IND_FACT|DIAS_FACT|  CSMO_D|   FE_FACT|
+-----------+------+------------+----------+--------------+------------+-----------------+-------------------+----------+-----------+---------+----------+---------------+---------+----+--------+---------+--------+----------+
|    1000006|201701|           8|         0|             0|           S|               32|2017-01-06 00:00:00|  10000006|      SU011|      B20|     TT001|              1|       15|   8|      Si|       32|0.250000|2017-01-06|
|    1000006|201702|          35|         0|             0|           S|               29|2017-02-04

In [10]:
join_2 = csm_sum.join(DXAA_GEO_BARRIOS, DXAA_GEO_BARRIOS.ID_BARRIO==join_1.ID_BARRIO)
join_2.show()

+-----------+------+------------+----------+--------------+------------+-----------------+-------------------+----------+-----------+---------+----------+---------------+---------+----+--------+---------+---------+----------+---------+---------+------------+---------------+---------+-------------------+
|ID_SERVICIO|ID_MES|CS_FACTURADO|CS_ANULADO|CS_REFACTURADO|IN_FACTURADO|CC_DIAS_FACTURADO|  FE_FACTURA_ULTIMA|ID_CLIENTE|TI_SERVICIO|TI_TARIFA|TI_TENSION|TI_TARIFA_GRUPO|ID_BARRIO|CSMO|IND_FACT|DIAS_FACT|   CSMO_D|   FE_FACT|ID_BARRIO|DE_BARRIO|ID_MUNICIPIO|ID_DEPARTAMENTO|ID_ESTADO|           FH_CARGA|
+-----------+------+------------+----------+--------------+------------+-----------------+-------------------+----------+-----------+---------+----------+---------------+---------+----+--------+---------+---------+----------+---------+---------+------------+---------------+---------+-------------------+
|    1000006|202106|           0|         0|             0|           S|             

In [11]:
join_3 = join_2.join(DXHS_LECTURAS, (col("DXHS_LECTURAS.ID_SERVICIO")==col("DXHS_FACTURACION_CONSUMOS.ID_SERVICIO"))\
                     &(col("DXHS_LECTURAS.ID_MES")==col("DXHS_FACTURACION_CONSUMOS.ID_MES")),"inner")\
.select(col('DXHS_FACTURACION_CONSUMOS.ID_SERVICIO').alias("NIS_RAD"),col('ID_CLIENTE').alias('cod_cli'),\
        'DXHS_FACTURACION_CONSUMOS.ID_MES',col('ID_APARATO').alias('num_apa'),col('DXHS_SERVICIOS.TI_TARIFA')\
        .alias('cod_tar'),col('DXHS_SERVICIOS.TI_TARIFA_GRUPO').alias('grupo_tar'),col('TI_SERVICIO').alias('tipo_suministro')\
       ,col('TI_TENSION').alias('tip_tension'),'CSMO','CSMO_D','DIAS_FACT','IND_FACT','FE_FACT',col('DXHS_SERVICIOS.ID_BARRIO')\
        .alias('cod_local'),col('ID_MUNICIPIO').alias('cod_munic'),col('ID_DEPARTAMENTO').alias('cod_depto')).cache()
join_3.show()

+-------+--------+------+------------+-------+---------+---------------+-----------+----+---------+---------+--------+----------+---------+---------+---------+
|NIS_RAD| cod_cli|ID_MES|     num_apa|cod_tar|grupo_tar|tipo_suministro|tip_tension|CSMO|   CSMO_D|DIAS_FACT|IND_FACT|   FE_FACT|cod_local|cod_munic|cod_depto|
+-------+--------+------+------------+-------+---------+---------------+-----------+----+---------+---------+--------+----------+---------+---------+---------+
|1000006|10000006|202106|  06B1013494|    B20|        1|          SU011|      TT001|   0| 0.000000|       28|      Si|2021-06-02|       15|      151|       26|
|1000014|10006316|202105|  006C208496|    B20|        1|          SU011|      TT001|  26| 0.866667|       30|      Si|2021-05-05|       15|      151|       26|
|1000019|10000019|201706|  06B1000170|    B20|        1|          SU011|      TT001|  61| 1.906250|       32|      Si|2017-06-05|       15|      151|       26|
|1000020|10000020|202105|  A17N841394|  

In [12]:
join_3_final = join_3.withColumn('CSMO_N', when(col('DIAS_FACT')!=0,((col('CSMO')*365)/col('DIAS_FACT'))/12).otherwise(0))
join_3_final.show()

+-------+--------+------+------------+-------+---------+---------------+-----------+----+---------+---------+--------+----------+---------+---------+---------+------------------+
|NIS_RAD| cod_cli|ID_MES|     num_apa|cod_tar|grupo_tar|tipo_suministro|tip_tension|CSMO|   CSMO_D|DIAS_FACT|IND_FACT|   FE_FACT|cod_local|cod_munic|cod_depto|            CSMO_N|
+-------+--------+------+------------+-------+---------+---------------+-----------+----+---------+---------+--------+----------+---------+---------+---------+------------------+
|1000006|10000006|202106|  06B1013494|    B20|        1|          SU011|      TT001|   0| 0.000000|       28|      Si|2021-06-02|       15|      151|       26|             0E-14|
|1000014|10006316|202105|  006C208496|    B20|        1|          SU011|      TT001|  26| 0.866667|       30|      Si|2021-05-05|       15|      151|       26| 26.36111111111083|
|1000019|10000019|201706|  06B1000170|    B20|        1|          SU011|      TT001|  61| 1.906250|      

In [43]:
join_3_final.count()

145676007

In [22]:
outputPath = f"s3a://{sourceBucket}/preprocessed_tables/consumosnormfact.parquet"

In [23]:
join_3_final.write.mode("overwrite").parquet(outputPath)

# CALCULOSNORM parquet

In [18]:
CONSUMO_NORM_FACT = spark.read.option("header", "true")\
    .parquet(f"s3a://data-lake/incremental/ConsumosNormFact.parquet").alias("CONSUMO_NORM_FACT")\
    .select(col("NIS_RAD").alias("nis_rad"), col("ID_MES").alias("id_mes"), col("CSMO").alias("csmo"),
            col("CSMO_D").alias("csm_d"), col("CSMO_N").alias("csmo_n")).filter(col("ID_MES") == 202301)
CONSUMO_NORM_FACT.show(1)

+-------+------+----+--------+-----------------+
|nis_rad|id_mes|csmo|   csm_d|           csmo_n|
+-------+------+----+--------+-----------------+
|1000842|202301|  88|2.750000|83.64583333333333|
+-------+------+----+--------+-----------------+
only showing top 1 row



In [19]:
#df_2 = join_3_final.select('nis_rad','id_mes','csmo','csmo_d','csmo_n')
df_2=CONSUMO_NORM_FACT
df_2.show()

+-------+------+----+---------+------------------+
|nis_rad|id_mes|csmo|    csm_d|            csmo_n|
+-------+------+----+---------+------------------+
|1000842|202301|  88| 2.750000| 83.64583333333333|
|1001087|202301| 310| 9.687500|294.66145833333333|
|1001111|202301|   2| 0.062500|  1.90104166666667|
|1001382|202301|  74| 2.312500| 70.33854166666667|
|1001404|202301| 171| 5.343750|162.53906250000000|
|1001416|202301| 193| 6.031250|183.45052083333333|
|1001686|202301| 196| 6.125000|186.30208333333333|
|1001992|202301| 116| 3.625000|110.26041666666667|
|1002011|202301|   0| 0.000000|             0E-14|
|1002528|202301|   1| 0.031250|  0.95052083333333|
|1003043|202301|  45| 1.406250| 42.77343750000000|
|1003164|202301|  85| 2.656250| 80.79427083333333|
|1003292|202301| 132| 4.125000|125.46875000000000|
|1003303|202301|  45| 1.406250| 42.77343750000000|
|1003487|202301|  94| 2.937500| 89.34895833333333|
|1003778|202301| 272| 8.500000|258.54166666666667|
|1004381|202301|  65| 2.031250|

In [20]:
df_2 = df_2.withColumn('consumoCero', when(col('csmo') == 0,1).otherwise(0))\
.withColumn('consumoCero_d', when(col('csmo') == 0,1).otherwise(0))\
.withColumn('consumoCero_n', when(col('csmo') == 0,1).otherwise(0)).cache()

In [21]:
df_2.show()

+-------+------+----+---------+------------------+-----------+-------------+-------------+
|nis_rad|id_mes|csmo|    csm_d|            csmo_n|consumoCero|consumoCero_d|consumoCero_n|
+-------+------+----+---------+------------------+-----------+-------------+-------------+
|1000842|202301|  88| 2.750000| 83.64583333333333|          0|            0|            0|
|1001087|202301| 310| 9.687500|294.66145833333333|          0|            0|            0|
|1001111|202301|   2| 0.062500|  1.90104166666667|          0|            0|            0|
|1001382|202301|  74| 2.312500| 70.33854166666667|          0|            0|            0|
|1001404|202301| 171| 5.343750|162.53906250000000|          0|            0|            0|
|1001416|202301| 193| 6.031250|183.45052083333333|          0|            0|            0|
|1001686|202301| 196| 6.125000|186.30208333333333|          0|            0|            0|
|1001992|202301| 116| 3.625000|110.26041666666667|          0|            0|            0|

In [22]:
df_2.select("id_mes").distinct().count()

1

In [ ]:
df_2.select("id_mes").orderBy(col("id_mes").desc()).show()

In [15]:
months = df_2.select("id_mes").distinct().orderBy(col("id_mes").asc()).withColumn("id_mes",df_2.id_mes.cast("int")).cache()
months.show()

+------+
|id_mes|
+------+
|201701|
|201702|
|201703|
|201704|
|201705|
|201706|
|201707|
|201708|
|201709|
|201710|
|201711|
|201712|
|201801|
|201802|
|201803|
|201804|
|201805|
|201806|
|201807|
|201808|
+------+
only showing top 20 rows



In [16]:
months_list = months.rdd.map(lambda x: x.id_mes).collect()


In [17]:
months_list

[201701,
 201702,
 201703,
 201704,
 201705,
 201706,
 201707,
 201708,
 201709,
 201710,
 201711,
 201712,
 201801,
 201802,
 201803,
 201804,
 201805,
 201806,
 201807,
 201808,
 201809,
 201810,
 201811,
 201812,
 201901,
 201902,
 201903,
 201904,
 201905,
 201906,
 201907,
 201908,
 201909,
 201910,
 201911,
 201912,
 202001,
 202002,
 202003,
 202004,
 202005,
 202006,
 202007,
 202008,
 202009,
 202010,
 202011,
 202012,
 202101,
 202102,
 202103,
 202104,
 202105,
 202106,
 202107,
 202108,
 202109,
 202110,
 202111,
 202112,
 202201,
 202202,
 202203,
 202204,
 202205,
 202206,
 202207,
 202208,
 202209,
 202210,
 202211,
 202212]

In [52]:
months_list=[201701,
 201702,
 201703,201704]
months_list

[201701, 201702, 201703, 201704]

In [17]:
#acumuladoConsumoCero
df_ant =  df_2.filter(df_2.id_mes==201701).withColumn("acumuladoConsumoCero",when(col("csmo")!=0,0)\
.when(col("csmo").isNull(),0).otherwise(lit(1))).alias("ant")
#acumuladoConsumoCero_d
df_ant =  df_ant.withColumn("acumuladoConsumoCero_d",when(col("csmo_d")!=0,0)\
.when(col("csmo").isNull(),0).otherwise(lit(1))).alias("ant")
#acumuladoConsumoCero_n
df_ant =  df_ant.withColumn("acumuladoConsumoCero_n",when(col("csmo_n")!=0,0)\
.when(col("csmo").isNull(),0).otherwise(lit(1))).alias("ant")
#consumoRepetido
df_ant =  df_ant.withColumn("consumoRepetido",lit(0)).alias("ant")
#consumoRepetido_d
df_ant =  df_ant.withColumn("consumoRepetido_d",lit(0)).alias("ant")
#consumoRepetido_n
df_ant =  df_ant.withColumn("consumoRepetido_n",lit(0)).alias("ant")
#acumuladoConsumoRepetido
df_ant =  df_ant.withColumn("acumuladoConsumoRepetido",lit(0)).alias("ant")
#acumuladoConsumoRepetido_d
df_ant =  df_ant.withColumn("acumuladoConsumoRepetido_d",lit(0)).alias("ant")
#acumuladoConsumoRepetido_n
df_ant =  df_ant.withColumn("acumuladoConsumoRepetido_n",lit(0)).alias("ant")
#\
#.withColumn("acumuladoConsumoCero_d",lit(0)).withColumn("acumuladoConsumoCero_n",lit(0))\
#.withColumn("consumoRepetido",lit(0)).withColumn("acumuladoConsumoCero_d",lit(0))
df_ant.show()

+-------+------+----+---------+------------------+-----------+-------------+-------------+--------------------+----------------------+----------------------+---------------+-----------------+-----------------+------------------------+--------------------------+--------------------------+
|nis_rad|id_mes|csmo|   csmo_d|            csmo_n|consumoCero|consumoCero_d|consumoCero_n|acumuladoConsumoCero|acumuladoConsumoCero_d|acumuladoConsumoCero_n|consumoRepetido|consumoRepetido_d|consumoRepetido_n|acumuladoConsumoRepetido|acumuladoConsumoRepetido_d|acumuladoConsumoRepetido_n|
+-------+------+----+---------+------------------+-----------+-------------+-------------+--------------------+----------------------+----------------------+---------------+-----------------+-----------------+------------------------+--------------------------+--------------------------+
|1000547|201701|  99| 3.093750| 94.10156250000000|          0|            0|            0|                   0|                     0

In [18]:
mySchema = StructType([StructField(c, StringType()) for c in df_ant.columns])

In [204]:
df_ant.columns

['nis_rad',
 'id_mes',
 'csmo',
 'csmo_d',
 'csmo_n',
 'consumoCero',
 'consumoCero_d',
 'consumoCero_n',
 'acumuladoConsumoCero',
 'acumuladoConsumoCero_d',
 'acumuladoConsumoCero_n',
 'consumoRepetido',
 'consumoRepetido_d',
 'consumoRepetido_n',
 'acumuladoConsumoRepetido',
 'acumuladoConsumoRepetido_d',
 'acumuladoConsumoRepetido_n']

In [230]:
len(columns_act)

17

In [19]:
columns_act =['act.nis_rad',
 'act.id_mes',
 'act.csmo',
 'act.csmo_d',
 'act.csmo_n',
 'act.consumoCero',
 'act.consumoCero_d',
 'act.consumoCero_n',
 'acumuladoConsumoCero',
 'acumuladoConsumoCero_d','acumuladoConsumoCero_n','consumoRepetido','consumoRepetido_d',
 'consumoRepetido_n','acumuladoConsumoRepetido','acumuladoConsumoRepetido_d',
 'acumuladoConsumoRepetido_n']

In [43]:
df_2.filter(df_2.id_mes==months_list[i]).show()

+-------+------+----+---------+------------------+-----------+-------------+-------------+
|nis_rad|id_mes|csmo|   csmo_d|            csmo_n|consumoCero|consumoCero_d|consumoCero_n|
+-------+------+----+---------+------------------+-----------+-------------+-------------+
|1000050|201702| 425|14.655172|445.76149425287333|          0|            0|            0|
|1000063|201702| 230| 7.931034|241.23563218390833|          0|            0|            0|
|1000713|201702| 215| 7.166667|217.98611111111083|          0|            0|            0|
|1001232|201702| 181| 6.241379|189.84195402298833|          0|            0|            0|
|1001331|201702|  75| 2.586207| 78.66379310344833|          0|            0|            0|
|1001691|201702|  98| 3.379310|102.78735632183917|          0|            0|            0|
|1001812|201702|  60| 2.068966| 62.93103448275833|          0|            0|            0|
|1001815|201702| 128| 4.413793|134.25287356321833|          0|            0|            0|

In [47]:
df_accumulative.show()

+-------+------+----+------+------+--------------------+----------------------+----------------------+---------------+-----------------+-----------------+------------------------+--------------------------+--------------------------+
|nis_rad|id_mes|csmo|csmo_d|csmo_n|acumuladoConsumoCero|acumuladoConsumoCero_d|acumuladoConsumoCero_n|consumoRepetido|consumoRepetido_d|consumoRepetido_n|acumuladoConsumoRepetido|acumuladoConsumoRepetido_d|acumuladoConsumoRepetido_n|
+-------+------+----+------+------+--------------------+----------------------+----------------------+---------------+-----------------+-----------------+------------------------+--------------------------+--------------------------+
+-------+------+----+------+------+--------------------+----------------------+----------------------+---------------+-----------------+-----------------+------------------------+--------------------------+--------------------------+



In [20]:
#print(df_ant.filter(col("ant.nis_rad")==1003515).show())
#df_accumulative = spark.createDataFrame(data=[],schema=mySchema)
#df_accumulative = df_accumulative.union(df_ant)
outputPath = f"s3a://{sourceBucket}/preprocessed_tables/calculosnorm.parquet"
df_ant.write.mode("overwrite").parquet(outputPath)

for i in range(1,len(months_list)): 
    
    df_act= df_2.filter(df_2.id_mes==months_list[i]).alias("act")
    #acumuladoConsumoCero
    join_act_ant = df_ant.join(df_act, col("ant.nis_rad")==col("act.nis_rad"))
    join_act_ant=join_act_ant.withColumn("acumuladoConsumoCero",when(col("ant.csmo")!=0,0).when(col("act.csmo")!=0,0)\
                           .when(col("act.csmo").isNull(),0).otherwise(col("ant.acumuladoConsumoCero")+1))
    #acumuladoConsumo_d
    join_act_ant=join_act_ant.withColumn("acumuladoConsumoCero_d",when(col("ant.csmo_d")!=0,0).when(col("act.csmo_d")!=0,0)\
                           .when(col("act.csmo_d").isNull(),0).otherwise(col("ant.acumuladoConsumoCero_d")+1))
    #acumuladoConsumo_n
    join_act_ant=join_act_ant.withColumn("acumuladoConsumoCero_n",when(col("ant.csmo_n")!=0,0).when(col("act.csmo_n")!=0,0)\
                           .when(col("act.csmo_n").isNull(),0).otherwise(col("ant.acumuladoConsumoCero_n")+1))
    #consumo repetido
    join_act_ant=join_act_ant.withColumn("consumoRepetido",when(col("ant.csmo").isNull(),0).when(col("act.csmo")==0,0)\
                           .when(col("act.csmo").isNull(),0).when(col("act.csmo")!=col("ant.csmo"),0).otherwise(lit(1)))
    #consumo repetido_d
    join_act_ant=join_act_ant.withColumn("consumoRepetido_d",when(col("ant.csmo_d").isNull(),0).when(col("act.csmo_d")==0,0)\
                           .when(col("act.csmo_d").isNull(),0).when(col("act.csmo_d")!=col("ant.csmo_d"),0).otherwise(lit(1)))
    #consumo repetido_d
    join_act_ant=join_act_ant.withColumn("consumoRepetido_n",when(col("ant.csmo_n").isNull(),0).when(col("act.csmo_n")==0,0)\
                           .when(col("act.csmo_n").isNull(),0).when(col("act.csmo_n")!=col("ant.csmo_n"),0).otherwise(lit(1)))
    #ConsumoAcumuladoRepetido
    join_act_ant=join_act_ant.withColumn("acumuladoConsumoRepetido",when(col("ant.csmo").isNull(),0).when(col("act.csmo")==0,0)\
                           .when(col("act.csmo").isNull(),0).when(col("act.csmo")!=col("ant.csmo"),0)\
                                         .when(col("ant.csmo")==0,2).otherwise(col("ant.acumuladoConsumoRepetido")+1))
    #ConsumoAcumuladoRepetido_d
    join_act_ant=join_act_ant.withColumn("acumuladoConsumoRepetido_d",when(col("ant.csmo_d").isNull(),0).when(col("act.csmo_d")==0,0)\
                           .when(col("act.csmo_d").isNull(),0).when(col("act.csmo_d")!=col("ant.csmo_d"),0)\
                                         .when(col("ant.csmo_d")==0,2).otherwise(col("ant.acumuladoConsumoRepetido_d")+1))
    #ConsumoAcumuladoRepetido_n
    join_act_ant=join_act_ant.withColumn("acumuladoConsumoRepetido_n",when(col("ant.csmo_n").isNull(),0).when(col("act.csmo_n")==0,0)\
                           .when(col("act.csmo_n").isNull(),0).when(col("act.csmo_n")!=col("ant.csmo_n"),0)\
                                         .when(col("ant.csmo_n")==0,2).otherwise(col("ant.acumuladoConsumoRepetido_n")+1))
 
    #df_act_final.show()
    df_act_final= join_act_ant.select(columns_act).alias("ant").cache()
    
    #df_accumulative = df_accumulative.union(df_act_final)
    outputPath = f"s3a://{sourceBucket}/preprocessed_tables/calculosnorm.parquet"
    df_act_final.write.mode("append").parquet(outputPath)
    df_ant = df_act_final.alias("ant").cache()
    print(months_list[i])
    #print(df_act_final.show())

201702
201703
201704
201705
201706
201707
201708
201709
201710
201711
201712
201801
201802
201803
201804
201805
201806
201807
201808
201809
201810
201811
201812
201901
201902
201903
201904
201905
201906
201907
201908
201909
201910
201911
201912
202001
202002
202003
202004
202005
202006
202007
202008
202009
202010
202011
202012
202101
202102
202103
202104
202105
202106
202107
202108
202109
202110
202111
202112
202201
202202
202203
202204
202205
202206
202207
202208
202209
202210
202211
202212


In [264]:
df_accumulative.select("id_mes").distinct().show(100)

+------+
|id_mes|
+------+
|201701|
|201702|
|201703|
|201704|
|201705|
|201706|
|201707|
|201708|
|201709|
|201710|
|201711|
|201712|
|201801|
|201802|
|201803|
|201804|
|201805|
|201806|
|201807|
|201808|
|201809|
|201810|
|201811|
|201812|
|201901|
|201902|
|201903|
|201904|
|201905|
|201906|
|201907|
|201908|
|201909|
|201910|
|201911|
|201912|
|202001|
|202002|
|202003|
|202004|
|202005|
|202006|
|202007|
|202008|
|202009|
|202010|
|202011|
|202012|
|202101|
|202102|
|202103|
|202104|
|202105|
|202106|
|202107|
|202108|
|202109|
|202110|
|202111|
|202112|
|202201|
|202202|
|202203|
|202204|
|202205|
|202206|
|202207|
|202208|
|202209|
|202210|
|202211|
|202212|
|202301|
+------+



In [229]:
df_accumulative.count()

5134583

In [ ]:
outputPath = f"s3a://{sourceBucket}/preprocessed_tables/calculosnorm.parquet"

In [ ]:
df_accumulative.write.mode("overwrite").parquet(outputPath)

# CalculosProm()

In [32]:
df = spark.read.option("header", "true").parquet(f"s3a://data-lake/incremental/ConsumosNormFact.parquet")\
.select("nis_rad","id_mes","csmo").alias("df")
df.show()


+-------+------+----+
|nis_rad|id_mes|csmo|
+-------+------+----+
|1000842|202301|  88|
|1001087|202301| 310|
|1001111|202301|   2|
|1001382|202301|  74|
|1001404|202301| 171|
|1001416|202301| 193|
|1001686|202301| 196|
|1001992|202301| 116|
|1002011|202301|   0|
|1002528|202301|   1|
|1003043|202301|  45|
|1003164|202301|  85|
|1003292|202301| 132|
|1003303|202301|  45|
|1003487|202301|  94|
|1003778|202301| 272|
|1004381|202301|  65|
|1004523|202301| 977|
|1004632|202301| 347|
|1004867|202301|  46|
+-------+------+----+
only showing top 20 rows



In [33]:
df_2 = spark.read.option("header", "true").parquet(f"s3a://data-lake/preprocessed_tables/consumosNormFact.parquet")\
.select("NIS_RAD","ID_MES","CSMO_N").filter((col("ID_MES")>=202202)&(col("ID_MES")<=202301)).alias("df2")
df_2.show()

+-------+------+------------------+
|NIS_RAD|ID_MES|            CSMO_N|
+-------+------+------------------+
|1000010|202206|133.83333333333333|
|1000014|202207|  9.50520833333333|
|1000020|202202| 81.43817204301083|
|1000036|202211| 77.05555555555583|
|1000045|202202|188.38709677419333|
|1000061|202210|203.41145833333333|
|1000062|202207|188.38709677419333|
|1000101|202205|159.18055555555583|
|1000115|202209|199.73611111111083|
|1000121|202206|150.05555555555583|
|1000131|202209| 28.38888888888917|
|1000135|202211|740.13888888888917|
|1000142|202202|148.15860215053750|
|1000163|202202| 16.68010752688167|
|1000173|202212| 97.54310344827583|
|1000191|202204| 23.31944444444417|
|1000238|202208| 18.64247311827917|
|1000312|202212|298.92241379310333|
|1000376|202208| 72.60752688172083|
|1000383|202205|744.68390804597667|
+-------+------+------------------+
only showing top 20 rows



In [34]:
df.count()

2309527

In [35]:
df_2.count()

24692495

In [39]:
calculosProm = df.union(df_2).select("nis_rad","id_mes","csmo")
calculosProm.show()

+-------+------+------------------+
|nis_rad|id_mes|              csmo|
+-------+------+------------------+
|1000842|202301| 88.00000000000000|
|1001087|202301|310.00000000000000|
|1001111|202301|  2.00000000000000|
|1001382|202301| 74.00000000000000|
|1001404|202301|171.00000000000000|
|1001416|202301|193.00000000000000|
|1001686|202301|196.00000000000000|
|1001992|202301|116.00000000000000|
|1002011|202301|             0E-14|
|1002528|202301|  1.00000000000000|
|1003043|202301| 45.00000000000000|
|1003164|202301| 85.00000000000000|
|1003292|202301|132.00000000000000|
|1003303|202301| 45.00000000000000|
|1003487|202301| 94.00000000000000|
|1003778|202301|272.00000000000000|
|1004381|202301| 65.00000000000000|
|1004523|202301|977.00000000000000|
|1004632|202301|347.00000000000000|
|1004867|202301| 46.00000000000000|
+-------+------+------------------+
only showing top 20 rows



In [6]:
calculosProm = df.filter((df.id_mes>=202202)&(df.id_mes<=202301)).select("nis_rad","id_mes","csmo")
calculosProm.show()

+-------+------+----+
|nis_rad|id_mes|csmo|
+-------+------+----+
|1000842|202301|  88|
|1001087|202301| 310|
|1001111|202301|   2|
|1001382|202301|  74|
|1001404|202301| 171|
|1001416|202301| 193|
|1001686|202301| 196|
|1001992|202301| 116|
|1002011|202301|   0|
|1002528|202301|   1|
|1003043|202301|  45|
|1003164|202301|  85|
|1003292|202301| 132|
|1003303|202301|  45|
|1003487|202301|  94|
|1003778|202301| 272|
|1004381|202301|  65|
|1004523|202301| 977|
|1004632|202301| 347|
|1004867|202301|  46|
+-------+------+----+
only showing top 20 rows



In [40]:
resultSeriesCache = calculosProm.groupBy("nis_rad").pivot(
    "id_mes").agg(first("csmo")).withColumn("prom3",lit(0)).withColumn("prom6",lit(0))\
.withColumn("prom12",lit(0)).cache()
resultSeriesCache = resultSeriesCache.withColumn("id_mes",lit(resultSeriesCache.columns[-4]))
resultSeriesCache.filter(col("nis_rad")==2029275).show(1000)

+-------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----+-----+------+------+
|nis_rad|           202202|           202203|           202204|           202205|           202206|           202207|           202208|           202209|           202210|           202211|           202212|           202301|prom3|prom6|prom12|id_mes|
+-------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----+-----+------+------+
|2029275|26.36111111111083|26.49193548387083|29.36781609195417|24.33333333333333|25.34722222222250|35.02525252525250|19.92816091954000|22.56720430107500|34.34139784946250|35.48611111111083|36.50000000000000|26.00000000000000|    0|    0|     0|

In [41]:
resultSeriesCache.show()

+-------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----+-----+------+------+
|nis_rad|            202202|            202203|            202204|            202205|            202206|            202207|            202208|            202209|            202210|            202211|            202212|            202301|prom3|prom6|prom12|id_mes|
+-------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----+-----+------+------+
|1007975| 26.49193548387083| 10.13888888888917|  6.29310344827583| 13.18055555555583|  2.94354838709667| 11.77419354838750|  6.86827956989250|  4.05555555555583|  1.96236559139750|  2.94354838709667|  3.14655

In [8]:
finalColumnsNames = ['nis_rad','csmo1','csmo2','csmo3', 'csmo4','csmo5',
                     'csmo6','csmo7','csmo8','csmo9','csmo10','csmo11','csmo12','prom3',
                     'prom6','prom12','id_mes']

finalColumnsOrder = ['nis_rad','id_mes','csmo1','csmo2','csmo3', 'csmo4','csmo5',
                     'csmo6','csmo7','csmo8','csmo9','csmo10','csmo11','csmo12','prom3',
                     'prom6','prom12']

In [ ]:
resultSeriesCache = resultSeriesCache.toDF(*finalColumnsNames).select(finalColumnsOrder)
resultSeriesCache.show()

In [ ]:
.select(finalOrderColumns).toDF(*finalColumnsNames)\
    .withColumn('AlturaPuntoConsumo', col('AlturaPuntoConsumo').cast('double')).cache()

In [ ]:
csm = resultSeriescache.columns[-16:]

In [41]:
df_ant = resultSeriesCache\
.withColumn("prom3m",(col("csmo10")+col("csmo11")+col("csmo12"))/3)\
.withColumn("prom6m",(col("csmo7")+col("csmo8")+col("csmo9")+col("csmo10")+col("csmo11")+col("csmo12"))/6)\
.withColumn("prom12m",(col("csmo1")+col("csmo2")+col("csmo3")+col("csmo4")\
                       +col("csmo5")+col("csmo6")+col("csmo7")+col("csmo8")+col("csmo9")\
                      +col("csmo10")+col("csmo11")+col("csmo12"))/12).alias("ant")

df_ant.show()

+-------+------+------+-----+-----+-----+-----+-----+-----+-----+-----+------+------+------+-----+-----+------+----------+----------+------------------+
|nis_rad|id_mes| csmo1|csmo2|csmo3|csmo4|csmo5|csmo6|csmo7|csmo8|csmo9|csmo10|csmo11|csmo12|prom3|prom6|prom12|    prom3m|    prom6m|           prom12m|
+-------+------+------+-----+-----+-----+-----+-----+-----+-----+-----+------+------+------+-----+-----+------+----------+----------+------------------+
|1000905|     0|201712|   28|   22|   19|   20|   18|   19|   17|   17|    22|    24|    28|   29|    0|     0| 24.666667| 21.166667|16828.833333333332|
|1001409|     0|201712|   36|   33|   34|    5|   28|   42|   38|   38|    33|    35|    37|   21|    0|     0| 35.000000| 37.166667|          16839.25|
|1001753|     0|201712|  102|   85|   97|   96|   92|  105|   87|   94|    83|    85|    82|   74|    0|     0| 83.333333| 89.333333|16893.333333333332|
|1004428|     0|201712|   81|   67|   82|   79|   69|   63|   74|   78|    75|    

In [5]:
months = df.select("id_mes").distinct().orderBy(col("id_mes").asc()).withColumn("id_mes",df.id_mes.cast("int")).cache()
months.show()
months_list = months.filter((col("id_mes")<=201802)).rdd.map(lambda x: x.id_mes).collect()

+------+
|id_mes|
+------+
|201701|
|201702|
|201703|
|201704|
|201705|
|201706|
|201707|
|201708|
|201709|
|201710|
|201711|
|201712|
|201801|
|201802|
|201803|
|201804|
|201805|
|201806|
|201807|
|201808|
+------+
only showing top 20 rows



In [48]:
df.filter((df.id_mes>=months_list[0])&(df.id_mes<=months_list[11])).select("nis_rad","id_mes","csmo").show()

+-------+------+----+
|nis_rad|id_mes|csmo|
+-------+------+----+
|1000060|201706|  79|
|1000103|201704| 146|
|1000180|201711| 197|
|1000181|201710|  18|
|1000212|201710| 336|
|1000221|201705| 482|
|1000258|201703|  91|
|1000271|201707|  83|
|1000302|201701|1548|
|1000305|201701| 209|
|1000332|201712|   0|
|1000348|201708| 128|
|1000354|201704| 148|
|1000368|201712|   0|
|1000386|201708|   0|
|1000403|201707|  17|
|1000410|201703| 111|
|1000413|201709| 283|
|1000432|201701|  17|
|1000468|201707| 158|
+-------+------+----+
only showing top 20 rows



In [51]:
df.filter((df.id_mes>=months_list[0])&(df.id_mes<=months_list[11])).select("id_mes").distinct().count()

12

In [9]:
months_list = months.rdd.map(lambda x: x.id_mes).collect()

In [13]:
#print(df_ant.filter(col("ant.nis_rad")==1003515).show())
#df_accumulative = spark.createDataFrame(data=[],schema=mySchema)
#df_accumulative = df_accumulative.union(df_ant)
#outputPath = f"s3a://{sourceBucket}/preprocessed_tables/calculosProm.parquet"
#df_ant.write.mode("overwrite").parquet(outputPath)

finalColumnsNames = ['nis_rad','csmo1','csmo2','csmo3', 'csmo4','csmo5',
                     'csmo6','csmo7','csmo8','csmo9','csmo10','csmo11','csmo12','prom3m',
                     'prom6m','prom12m','id_mes']

finalColumnsOrder = ['nis_rad','id_mes','csmo1','csmo2','csmo3', 'csmo4','csmo5',
                     'csmo6','csmo7','csmo8','csmo9','csmo10','csmo11','csmo12','prom3m',
                     'prom6m','prom12m']

outputPath = f"s3a://data-lake/preprocessed_tables/calculosProm.parquet"

for i in range(len(months_list)-11): 
    
    
    calculosProm= df.filter((df.id_mes>=months_list[i])&(df.id_mes<=months_list[i+11])).select("nis_rad","id_mes","csmo").alias("act")
    
    resultSeriesCache = calculosProm.groupBy("nis_rad").pivot(
    "id_mes").agg(first("csmo")).withColumn("prom3m",lit(0)).withColumn("prom6m",lit(0))\
    .withColumn("prom12m",lit(0)).cache()
    
    
    resultSeriesCache = resultSeriesCache.withColumn("id_mes",lit(resultSeriesCache.columns[-4]))
    
    
    resultSeriesCache = resultSeriesCache.toDF(*finalColumnsNames).select(finalColumnsOrder)
    
    
    df_final = resultSeriesCache.withColumn("prom3m",(col("csmo10")+col("csmo11")+col("csmo12"))/3)\
    .withColumn("prom6m",(col("csmo7")+col("csmo8")+col("csmo9")+col("csmo10")+col("csmo11")+col("csmo12"))/6)\
    .withColumn("prom12m",(col("csmo1")+col("csmo2")+col("csmo3")+col("csmo4")\
                       +col("csmo5")+col("csmo6")+col("csmo7")+col("csmo8")+col("csmo9")\
                      +col("csmo10")+col("csmo11")+col("csmo12"))/12)
    
    print(months_list[i],months_list[i+11])
    #df_final.show()
    
    if i == 0:
        
        df_final.write.mode("overwrite").parquet(outputPath)
        print("=========")
    else:
        
        df_final.write.mode("append").parquet(outputPath)
        print("==========")
        

201701 201712
201702 201801
201703 201802
201704 201803
201705 201804
201706 201805
201707 201806
201708 201807
201709 201808
201710 201809
201711 201810
201712 201811
201801 201812
201802 201901
201803 201902
201804 201903
201805 201904
201806 201905
201807 201906
201808 201907
201809 201908
201810 201909
201811 201910
201812 201911
201901 201912
201902 202001
201903 202002
201904 202003
201905 202004
201906 202005
201907 202006
201908 202007
201909 202008
201910 202009
201911 202010
201912 202011
202001 202012
202002 202101
202003 202102
202004 202103
202005 202104
202006 202105
202007 202106
202008 202107
202009 202108
202010 202109
202011 202110
202012 202111
202101 202112
202102 202201
202103 202202
202104 202203
202105 202204
202106 202205
202107 202206
202108 202207
202109 202208
202110 202209
202111 202210
202112 202211
202201 202212


In [12]:
df_final.show()

+-------+------+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+------+----------+----------+----------+
|nis_rad|id_mes|csmo1|csmo2|csmo3|csmo4|csmo5|csmo6|csmo7|csmo8|csmo9|csmo10|csmo11|csmo12|    prom3m|    prom6m|   prom12m|
+-------+------+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+------+----------+----------+----------+
|1000905|201712|   28|   22|   19|   20|   18|   19|   17|   17|   22|    24|    28|    29| 27.000000| 22.833333| 21.916667|
|1001409|201712|   36|   33|   34|    5|   28|   42|   38|   38|   33|    35|    37|    21| 31.000000| 33.666667| 31.666667|
|1001753|201712|  102|   85|   97|   96|   92|  105|   87|   94|   83|    85|    82|    74| 80.333333| 84.166667| 90.166667|
|1004428|201712|   81|   67|   82|   79|   69|   63|   74|   78|   75|    74|    79|    80| 77.666667| 76.666667| 75.083333|
|1007802|201712|   98|   67|   69|   82|   78|   99|   86|   75|   91|    94|    83|    85| 87.333333| 85.666667| 83.916667|


# HECHO CONSUMO

In [33]:
CONSUMO_NORM_FACT.columns

['NIS_RAD',
 'cod_cli',
 'ID_MES',
 'num_apa',
 'cod_tar',
 'grupo_tar',
 'tipo_suministro',
 'tip_tension',
 'CSMO',
 'CSMO_D',
 'DIAS_FACT',
 'IND_FACT',
 'FE_FACT',
 'cod_local',
 'cod_munic',
 'cod_depto',
 'CSMO_N']

In [45]:
HechoConsumo.printSchema()

root
 |-- LlaveConsumoNormalizado: long (nullable = true)
 |-- DiasFacturadosCN: decimal(10,0) (nullable = true)
 |-- EnergiaActivakwhcn: decimal(12,0) (nullable = true)
 |-- csmo_d: decimal(18,6) (nullable = true)
 |-- csmo_n: decimal(30,14) (nullable = true)
 |-- LlavePuntoConsumo: integer (nullable = true)
 |-- LlaveFecha: integer (nullable = true)
 |-- ConsumoCeroCN: integer (nullable = true)
 |-- AcumuladoConsumoCeroCN: integer (nullable = true)
 |-- ConsumoRepetidoCN: integer (nullable = true)
 |-- AcumuladoConsumoRepetidoCN: integer (nullable = true)
 |-- PromedioActiva3CN: decimal(20,6) (nullable = true)
 |-- PromedioActiva6CN: decimal(23,6) (nullable = true)
 |-- PromedioActiva12CN: decimal(29,6) (nullable = true)
 |-- FechaCargaRegistro: date (nullable = true)



In [3]:
HechoConsumo = (
    spark.read
    .format("jdbc")
    .option("url", "jdbc:postgresql://postgres/smartrecovery")
    .option("dbtable", "public.\"HechoConsumoNormalizado\"")
    .option("user", "admin")
    .option("password", "secret")
    .option("driver", "org.postgresql.Driver")
    .load().alias("HechoConsumo"))

In [ ]:
HechoConsumo.groupBy(col("FechaCargaRegistro")).count().orderBy(col("count").desc()).show(200)#filter(col("LlaveFecha")>20221232).count()

In [5]:
from datetime import date
today = date.today()
today

datetime.date(2023, 5, 2)

In [8]:
print(lit(today))

Column<'DATE '2023-05-02''>


In [5]:
CONSUMO_NORM_FACT = spark.read.option("header", "true")\
    .parquet(f"s3a://data-lake/incremental/ConsumosNormFact.parquet").alias("CONSUMO_NORM_FACT")\
    .select("NIS_RAD", "ID_MES", col("DIAS_FACT").alias("diasFacturadosCN"), col("CSMO").alias("EnergiaActivakWhCN"),
            "CSMO_D", "CSMO_N", "FE_FACT").filter(col("ID_MES") == 202301)
CONSUMO_NORM_FACT.show(1)

+-------+------+----------------+------------------+--------+-----------------+----------+
|NIS_RAD|ID_MES|diasFacturadosCN|EnergiaActivakWhCN|  CSMO_D|           CSMO_N|   FE_FACT|
+-------+------+----------------+------------------+--------+-----------------+----------+
|1000842|202301|              32|                88|2.750000|83.64583333333333|2023-01-03|
+-------+------+----------------+------------------+--------+-----------------+----------+
only showing top 1 row



In [ ]:
CONSUMO_NORM_FACT = spark.read.option("header", "true")\
.parquet(f"s3a://data-lake/preprocessed_tables/consumosnormfact.parquet").alias("CONSUMO_NORM_FACT")\
.select("NIS_RAD","ID_MES",col("DIAS_FACT").alias("diasFacturadosCN"),col("CSMO").alias("EnergiaActivakWhCN"),\
        "CSMO_D","CSMO_N","FE_FACT")
CONSUMO_NORM_FACT.show(1)

In [32]:
CALCULOS_NORM = spark.read.option("header", "true")\
    .parquet(f"s3a://data-lake/incremental/CalculosNorm.parquet").alias("CALCULOS_NORM")\
    .select("nis_rad", "id_mes", col("consumoCero").alias("ConsumoCeroCN"),
            col("acumuladoConsumoCero").alias("AcumuladoConsumoCeroCN"), col(
            "consumoRepetido").alias("ConsumoRepetidoCN"),
            col("acumuladoConsumoRepetido").alias("AcumuladoConsumoRepetidoCN"))#.filter(col("id_mes") == 202301)
CALCULOS_NORM.show(100)

+-------+------+-------------+----------------------+-----------------+--------------------------+
|nis_rad|id_mes|ConsumoCeroCN|AcumuladoConsumoCeroCN|ConsumoRepetidoCN|AcumuladoConsumoRepetidoCN|
+-------+------+-------------+----------------------+-----------------+--------------------------+
|1000001|202301|            0|                     0|                0|                         0|
|1000005|202301|            0|                     0|                0|                         0|
|1000010|202301|            0|                     0|                0|                         0|
|1000012|202301|            0|                     0|                0|                         0|
|1000018|202301|            0|                     0|                0|                         0|
|1000021|202301|            0|                     0|                0|                         0|
|1000022|202301|            0|                     0|                0|                         0|
|1000024|2

In [4]:
CALCULOS_NORM = spark.read.option("header", "true")\
.parquet(f"s3a://data-lake/preprocessed_tables/calculosnorm.parquet").alias("CALCULOS_NORM")\
.select("nis_rad","id_mes",col("consumoCero").alias("ConsumoCeroCN"),
        col("acumuladoConsumoCero").alias("AcumuladoConsumoCeroCN"),col("consumoRepetido").alias("ConsumoRepetidoCN"),\
        col("acumuladoConsumoRepetido").alias("AcumuladoConsumoRepetidoCN"))
CALCULOS_NORM.show()



+-------+------+-------------+----------------------+-----------------+--------------------------+
|nis_rad|id_mes|ConsumoCeroCN|AcumuladoConsumoCeroCN|ConsumoRepetidoCN|AcumuladoConsumoRepetidoCN|
+-------+------+-------------+----------------------+-----------------+--------------------------+
|1000305|201709|            0|                     0|                0|                         0|
|1000369|201709|            0|                     0|                0|                         0|
|1000542|201709|            0|                     0|                0|                         0|
|1000600|201709|            0|                     0|                0|                         0|
|1000638|201709|            0|                     0|                0|                         0|
|1000698|201709|            0|                     0|                0|                         0|
|1000809|201709|            1|                     1|                0|                         0|
|1000964|2

In [11]:
CALCULOS_PROM = spark.read.option("header", "true")\
    .parquet(f"s3a://data-lake/incremental/CalculosProm.parquet").alias("CALCULOS_PROM")\
    .select("nis_rad", "id_mes", col("prom3m").alias("PromedioActiva3CN"), col("prom6m")
            .alias("PromedioActiva6CN"), col("prom12m").alias("PromedioActiva12CN")).filter(col("id_mes") == 202301)
CALCULOS_PROM.show(1)

+-------+------+-------------------+-------------------+------------------+
|nis_rad|id_mes|  PromedioActiva3CN|  PromedioActiva6CN|PromedioActiva12CN|
+-------+------+-------------------+-------------------+------------------+
|1019473|202301|63.0972222222222233|54.6064068100358333|    79.04475973819|
+-------+------+-------------------+-------------------+------------------+
only showing top 1 row



In [15]:

x = spark.read.option("header", "true").parquet(f"s3a://data-lake/incremental/ConsumosNormFact.parquet")\
    .select("nis_rad", "id_mes", "csmo")#.filter((col("ID_MES") >= 202202) & (col("ID_MES") <= period_to_update.strftime('%Y%m'))).alias("df")
x.show()

+-------+------+----+
|nis_rad|id_mes|csmo|
+-------+------+----+
|1000842|202301|  88|
|1001087|202301| 310|
|1001111|202301|   2|
|1001382|202301|  74|
|1001404|202301| 171|
|1001416|202301| 193|
|1001686|202301| 196|
|1001992|202301| 116|
|1002011|202301|   0|
|1002528|202301|   1|
|1003043|202301|  45|
|1003164|202301|  85|
|1003292|202301| 132|
|1003303|202301|  45|
|1003487|202301|  94|
|1003778|202301| 272|
|1004381|202301|  65|
|1004523|202301| 977|
|1004632|202301| 347|
|1004867|202301|  46|
+-------+------+----+
only showing top 20 rows



In [5]:
CALCULOS_PROM = spark.read.option("header", "true")\
.parquet(f"s3a://data-lake/preprocessed_tables/calculosProm.parquet").alias("CALCULOS_PROM")\
.select("nis_rad","id_mes",col("prom3m").alias("PromedioActiva3CN"),col("prom6m").alias("PromedioActiva6CN")\
        ,col("prom12m").alias("PromedioActiva12CN"))

CALCULOS_PROM.show()

+-------+------+-----------------+-----------------+------------------+
|nis_rad|id_mes|PromedioActiva3CN|PromedioActiva6CN|PromedioActiva12CN|
+-------+------+-----------------+-----------------+------------------+
|1011098|202212|        87.333333|        89.166667|        124.916667|
|1021058|202212|       136.000000|       131.000000|        126.833333|
|1023702|202212|        45.666667|        46.166667|         47.250000|
|1027176|202212|         4.000000|         2.666667|          2.166667|
|1028447|202212|       111.000000|       116.166667|         98.583333|
|1028757|202212|        83.333333|        81.666667|         78.500000|
|2000774|202212|         0.000000|             null|              null|
|2005160|202212|         0.000000|         0.000000|         25.500000|
|2008229|202212|        91.000000|        92.833333|         94.583333|
|2016941|202212|        29.333333|        24.000000|         28.833333|
|2024923|202212|        91.666667|        87.666667|         93.

In [6]:

SUMCON = spark.read.option("header", "true").parquet(f"s3a://data-lake/DWH_datadwh/SUMCON.parquet").alias("CALCULOS_PROM")



In [7]:
DIMENSION_PUNTO_CONSUMO = (
    spark.read
    .format("jdbc")
    .option("url", "jdbc:postgresql://postgres/smartrecovery")
    .option("dbtable", "public.DimensionPuntoconsumo")
    .option("user", "admin")
    .option("password", "secret")
    .load().select("LlavePuntoConsumo","CodigoPuntoConsumo")
)

In [24]:
DIMENSION_FECHA = (
    spark.read
    .format("jdbc")
    .option("url", "jdbc:postgresql://postgres/smartrecovery")
    .option("dbtable", "public.DimensionFecha")
    .option("user", "admin")
    .option("password", "secret")
    .load().alias("DIMENSION_FECHA"))
DIMENSION_FECHA.show()

+----------+-----------+----------------+-----------------+----------+--------------+-----------+-----------+--------------+-----------+-----------------+----------------+--------+------------------------+---------------------+---------------------------+--------------------------+-----------------------------+--------------------------+--------------------------------+-------------------------------+--------------------+---------------------+-----------------+------------------+------------+-----------------------+------------------+-------------+-------+-------+
|LlaveFecha|CodigoFecha|TextoInglesFecha|TextoEspañolFecha|  DMAFecha|DiaSemanaFecha|DiaMesFecha|DiaAñoFecha|SemanaAñoFecha|MesAñoFecha|TrimestreAñoFecha|SemestreAñoFecha|AñoFecha|CalendarioAñoSemanaFecha|CalendarioAñoMesFecha|CalendarioAñoTrimestreFecha|CalendarioAñoSemestreFecha|CalendarioAñoSemanaTextoFecha|CalendarioAñoMesTextoFecha|CalendarioAñoTrimestreTextoFecha|CalendarioAñoSemestreTextoFecha|DiaSemanaInglesFecha|DiaS

In [45]:
join_1 = CONSUMO_NORM_FACT.join(
    SUMCON, (CONSUMO_NORM_FACT.NIS_RAD == SUMCON.NIS_RAD)).drop(SUMCON.NIS_RAD)
join_1.count()

In [52]:
len(join_1.columns)

12

130

In [26]:
join_2 = join_1.join(CALCULOS_NORM, (CALCULOS_NORM.nis_rad == CONSUMO_NORM_FACT.NIS_RAD) &
                     (CALCULOS_NORM.id_mes == CONSUMO_NORM_FACT.ID_MES)).drop("CALCULOS_NORM.nis_rad", "CALCULOS_NORM.id_mes")


In [10]:
join_2.count()

+-------+------+----------------+------------------+---------+------------------+-------+------+-------------+----------------------+-----------------+--------------------------+
|NIS_RAD|ID_MES|diasFacturadosCN|EnergiaActivakWhCN|   CSMO_D|            CSMO_N|nis_rad|id_mes|ConsumoCeroCN|AcumuladoConsumoCeroCN|ConsumoRepetidoCN|AcumuladoConsumoRepetidoCN|
+-------+------+----------------+------------------+---------+------------------+-------+------+-------------+----------------------+-----------------+--------------------------+
|1000006|202106|              28|                 0| 0.000000|             0E-14|1000006|202106|            1|                     3|                0|                         0|
|1000019|201706|              32|                61| 1.906250| 57.98177083333333|1000019|201706|            0|                     0|                0|                         0|
|1000020|202105|              30|                89| 2.966667| 90.23611111111083|1000020|202105|         

In [11]:
join_1.printSchema()

root
 |-- NIS_RAD: decimal(7,0) (nullable = true)
 |-- ID_MES: decimal(6,0) (nullable = true)
 |-- diasFacturadosCN: decimal(10,0) (nullable = true)
 |-- EnergiaActivakWhCN: decimal(12,0) (nullable = true)
 |-- CSMO_D: decimal(18,6) (nullable = true)
 |-- CSMO_N: decimal(30,14) (nullable = true)
 |-- nis_rad: decimal(7,0) (nullable = true)
 |-- id_mes: decimal(6,0) (nullable = true)
 |-- ConsumoCeroCN: integer (nullable = true)
 |-- AcumuladoConsumoCeroCN: integer (nullable = true)
 |-- ConsumoRepetidoCN: integer (nullable = true)
 |-- AcumuladoConsumoRepetidoCN: integer (nullable = true)



In [27]:
join_3 = join_2.join(CALCULOS_PROM, (CALCULOS_PROM.nis_rad == CONSUMO_NORM_FACT.NIS_RAD) &
                     (CALCULOS_PROM.id_mes == CONSUMO_NORM_FACT.ID_MES)).drop("CALCULOS_PROM.nis_rad", "CALCULOS_PROM.id_mes")
join_3.count()

In [ ]:
join_4 = join_3.join(DIMENSION_PUNTO_CONSUMO,
                     (DIMENSION_PUNTO_CONSUMO.CodigoPuntoConsumo == CONSUMO_NORM_FACT.NIS_RAD)).drop("DIMENSION_PUNTO_CONSUMO.nis_rad", "DIMENSION_PUNTO_CONSUMO.id_mes")
join_4.count()

In [ ]:
join_5 = join_4.join(
    DIMENSION_FECHA, (DIMENSION_FECHA.CodigoFecha == join_4.FE_FACT))
join_5.count()

In [ ]:
today = date.today()

df_final = join_5.withColumn("FechaCargaRegistro", lit(today))\
    .withColumn("LlaveConsumoNormalizado", monotonically_increasing_id()).select("LlaveConsumoNormalizado", "DiasFacturadosCN", "EnergiaActivakwhcn", "csmo_d", "csmo_n", "LlavePuntoConsumo", "LlaveFecha", "ConsumoCeroCN",
                                                                                 "AcumuladoConsumoCeroCN", "ConsumoRepetidoCN", "AcumuladoConsumoRepetidoCN", "PromedioActiva3CN",
                                                                                 "PromedioActiva6CN", "PromedioActiva12CN", "FechaCargaRegistro")
df_final.count()

In [ ]:
DIMENSION_PUNTO_CONSUMO.printSchema()

In [31]:
DIMENSION_PUNTO_CONSUMO.printSchema()

root
 |-- LlavePuntoConsumo: integer (nullable = true)
 |-- CodigoPuntoConsumo: string (nullable = true)
 |-- FechaCreacionPuntoconsumo: timestamp (nullable = true)
 |-- TipoEstadoPuntoConsumo: string (nullable = true)
 |-- NombreEstadoPuntoConsumo: string (nullable = true)
 |-- LatitudPuntoConsumo: string (nullable = true)
 |-- LongitudPuntoConsumo: string (nullable = true)
 |-- AlturaPuntoConsumo: string (nullable = true)
 |-- DireccionPuntoConsumo: string (nullable = true)
 |-- CodigoTipoPuntoConsumo: string (nullable = true)
 |-- TipoPuntoConsumo: string (nullable = true)
 |-- CodigoTarifa: string (nullable = true)
 |-- CargaContratadaPuntoConsumo: string (nullable = true)
 |-- FechaFinRegistro: date (nullable = true)
 |-- Peaje: string (nullable = true)
 |-- Mercado: string (nullable = true)
 |-- EstadoRegistro: integer (nullable = true)
 |-- VisibleFacturacion: integer (nullable = true)
 |-- FechaInicioRegistro: timestamp (nullable = true)



In [24]:
CALCULOS_NORM.printSchema()

root
 |-- nis_rad: decimal(7,0) (nullable = true)
 |-- id_mes: decimal(6,0) (nullable = true)
 |-- csmo: decimal(12,0) (nullable = true)
 |-- csmo_d: decimal(18,6) (nullable = true)
 |-- csmo_n: decimal(30,14) (nullable = true)
 |-- consumoCero: integer (nullable = true)
 |-- consumoCero_d: integer (nullable = true)
 |-- consumoCero_n: integer (nullable = true)
 |-- acumuladoConsumoCero: integer (nullable = true)
 |-- acumuladoConsumoCero_d: integer (nullable = true)
 |-- acumuladoConsumoCero_n: integer (nullable = true)
 |-- consumoRepetido: integer (nullable = true)
 |-- consumoRepetido_d: integer (nullable = true)
 |-- consumoRepetido_n: integer (nullable = true)
 |-- acumuladoConsumoRepetido: integer (nullable = true)
 |-- acumuladoConsumoRepetido_d: integer (nullable = true)
 |-- acumuladoConsumoRepetido_n: integer (nullable = true)



# dimensionFecha carga

In [40]:

DIM_FECHA = spark.read.option("header", "true").parquet(f"s3a://data-lake/utils/dimension_fecha.parquet")
DIM_FECHA.show()

+----------+-----------+----------------+-----------------+----------+--------------+-----------+-----------+--------------+-----------+-----------------+----------------+--------+------------------------+---------------------+---------------------------+--------------------------+-----------------------------+--------------------------+--------------------------------+-------------------------------+--------------------+---------------------+-----------------+------------------+------------+-----------------------+------------------+-------------+-------+-------+
|LlaveFecha|CodigoFecha|TextoInglesFecha|TextoEspañolFecha|  DMAFecha|DiaSemanaFecha|DiaMesFecha|DiaAñoFecha|SemanaAñoFecha|MesAñoFecha|TrimestreAñoFecha|SemestreAñoFecha|AñoFecha|CalendarioAñoSemanaFecha|CalendarioAñoMesFecha|CalendarioAñoTrimestreFecha|CalendarioAñoSemestreFecha|CalendarioAñoSemanaTextoFecha|CalendarioAñoMesTextoFecha|CalendarioAñoTrimestreTextoFecha|CalendarioAñoSemestreTextoFecha|DiaSemanaInglesFecha|DiaS

In [41]:
(
    DIM_FECHA.write
    .format("jdbc")
    .option("url", "jdbc:postgresql://postgres/smartrecovery")
    .option("dbtable", "public.\"DimensionFecha\"")
    .option("user", "admin")
    .option("password", "secret")
    .option("driver", "org.postgresql.Driver")
    .option("truncate", True) \
    .mode("overwrite")
    .save()
)

In [22]:
x = (
    spark.read
    .format("jdbc")
    .option("url", "jdbc:postgresql://postgres/smartrecovery")
    .option("dbtable", "public.dimensionFecha")
    .option("user", "admin")
    .option("password", "secret")
    .load())
x.show()

+----------+-----------+----------------+-----------------+----------+--------------+-----------+-----------+--------------+-----------+-----------------+----------------+--------+------------------------+---------------------+---------------------------+--------------------------+-----------------------------+--------------------------+--------------------------------+-------------------------------+--------------------+---------------------+-----------------+------------------+------------+-----------------------+------------------+-------------+-------+-------+
|LlaveFecha|CodigoFecha|TextoInglesFecha|TextoEspañolFecha|  DMAFecha|DiaSemanaFecha|DiaMesFecha|DiaAñoFecha|SemanaAñoFecha|MesAñoFecha|TrimestreAñoFecha|SemestreAñoFecha|AñoFecha|CalendarioAñoSemanaFecha|CalendarioAñoMesFecha|CalendarioAñoTrimestreFecha|CalendarioAñoSemestreFecha|CalendarioAñoSemanaTextoFecha|CalendarioAñoMesTextoFecha|CalendarioAñoTrimestreTextoFecha|CalendarioAñoSemestreTextoFecha|DiaSemanaInglesFecha|DiaS

# Llaves hechos (facteless fact atble)

In [3]:
SUMCON = spark.read.option("header", "true").parquet(
    f"s3a://data-lake/DWH_datadwh/SUMCON.parquet").alias("SUMCOM").select("NIS_RAD","EST_SUM","COD_TAR")

SUMCON.printSchema()

root
 |-- NIS_RAD: decimal(7,0) (nullable = true)
 |-- EST_SUM: string (nullable = true)
 |-- COD_TAR: string (nullable = true)



In [4]:
DXES_SERVICIOS = spark.read.option("header", "true").parquet(
    f"s3a://data-lake/DWH_datadwh/DXES_SERVICIOS.parquet").alias("DXES_SERVICIOS").select("ID_SERVICIO","ID_BARRIO","ID_CNAE"\
                           ,"ID_OFICINA","ID_LECTURA_CENTRO","ID_LECTURA_RUTA","ID_LECTURA_ITINERARIO")
DXES_SERVICIOS.printSchema()

root
 |-- ID_SERVICIO: decimal(7,0) (nullable = true)
 |-- ID_BARRIO: decimal(8,0) (nullable = true)
 |-- ID_CNAE: string (nullable = true)
 |-- ID_OFICINA: decimal(4,0) (nullable = true)
 |-- ID_LECTURA_CENTRO: decimal(4,0) (nullable = true)
 |-- ID_LECTURA_RUTA: decimal(4,0) (nullable = true)
 |-- ID_LECTURA_ITINERARIO: decimal(4,0) (nullable = true)



In [5]:
join_1 = SUMCON.join(DXES_SERVICIOS, SUMCON.NIS_RAD == DXES_SERVICIOS.ID_SERVICIO )
join_1.show(1)

+-------+-------+-------+-----------+---------+-------+----------+-----------------+---------------+---------------------+
|NIS_RAD|EST_SUM|COD_TAR|ID_SERVICIO|ID_BARRIO|ID_CNAE|ID_OFICINA|ID_LECTURA_CENTRO|ID_LECTURA_RUTA|ID_LECTURA_ITINERARIO|
+-------+-------+-------+-----------+---------+-------+----------+-----------------+---------------+---------------------+
|2099214|  EC023|    B20|    2099214|     1163|   9005|      2302|             2401|              2|                 2300|
+-------+-------+-------+-----------+---------+-------+----------+-----------------+---------------+---------------------+
only showing top 1 row



In [6]:
DXAA_GEO_BARRIOS = spark.read.option("header", "true").parquet(
    f"s3a://data-lake/DWH_datadwh/DXAA_GEO_BARRIOS.parquet").alias("DXAA_GEO_BARRIOS").select("ID_BARRIO","ID_ESTADO","ID_DEPARTAMENTO","ID_MUNICIPIO")
DXAA_GEO_BARRIOS.printSchema()

root
 |-- ID_BARRIO: decimal(9,0) (nullable = true)
 |-- ID_ESTADO: decimal(9,0) (nullable = true)
 |-- ID_DEPARTAMENTO: decimal(9,0) (nullable = true)
 |-- ID_MUNICIPIO: decimal(9,0) (nullable = true)



In [7]:
join_2 = join_1.join(DXAA_GEO_BARRIOS, DXAA_GEO_BARRIOS.ID_BARRIO == join_1.ID_BARRIO).drop(DXAA_GEO_BARRIOS.ID_BARRIO)
join_2.count()

2842632

In [8]:
DIMENSION_PUNTO_CONSUMO = (
    spark.read
    .format("jdbc")
    .option("url", "jdbc:postgresql://postgres/smartrecovery")
    .option("dbtable", "public.\"DimensionPuntoConsumo\"")
    .option("user", "admin")
    .option("password", "secret")
    .option("driver", "org.postgresql.Driver")
    .load().alias("DIMENSION_PUNTO_CONSUMO")).select("CodigoPuntoConsumo")
DIMENSION_PUNTO_CONSUMO.printSchema()

root
 |-- CodigoPuntoConsumo: string (nullable = true)



In [9]:
join_3 = join_2.join(DIMENSION_PUNTO_CONSUMO,join_2.NIS_RAD==DIMENSION_PUNTO_CONSUMO.CodigoPuntoConsumo).dropna()
join_3.count()

2842632

In [25]:
DIMENSION_GEOGRAFIA = (
    spark.read
    .format("jdbc")
    .option("url", "jdbc:postgresql://postgres/smartrecovery")
    .option("dbtable", "public.\"DimensionGeografia\"")
    .option("user", "admin")
    .option("password", "secret")
    .option("driver", "org.postgresql.Driver")
    .load().alias("DIMENSION_GEOGRAFIA")).select("LlaveGeografia","CodigoDepartamentoGeografia",\
                                                 "CodigoMunicipioGeografia","CodigoLocalidadZonaGeografia").dropna()
DIMENSION_GEOGRAFIA.printSchema()

root
 |-- LlaveGeografia: integer (nullable = true)
 |-- CodigoDepartamentoGeografia: string (nullable = true)
 |-- CodigoMunicipioGeografia: string (nullable = true)
 |-- CodigoLocalidadZonaGeografia: decimal(10,0) (nullable = true)



In [11]:
join_3.printSchema()

root
 |-- NIS_RAD: decimal(7,0) (nullable = true)
 |-- EST_SUM: string (nullable = true)
 |-- COD_TAR: string (nullable = true)
 |-- ID_SERVICIO: decimal(7,0) (nullable = true)
 |-- ID_BARRIO: decimal(8,0) (nullable = true)
 |-- ID_CNAE: string (nullable = true)
 |-- ID_OFICINA: decimal(4,0) (nullable = true)
 |-- ID_LECTURA_CENTRO: decimal(4,0) (nullable = true)
 |-- ID_LECTURA_RUTA: decimal(4,0) (nullable = true)
 |-- ID_LECTURA_ITINERARIO: decimal(4,0) (nullable = true)
 |-- ID_ESTADO: decimal(9,0) (nullable = true)
 |-- ID_DEPARTAMENTO: decimal(9,0) (nullable = true)
 |-- ID_MUNICIPIO: decimal(9,0) (nullable = true)
 |-- CodigoPuntoConsumo: string (nullable = true)



In [12]:
join_3.filter(col("nis_rad")==1000012).show()

+-------+-------+-------+-----------+---------+-------+----------+-----------------+---------------+---------------------+---------+---------------+------------+------------------+
|NIS_RAD|EST_SUM|COD_TAR|ID_SERVICIO|ID_BARRIO|ID_CNAE|ID_OFICINA|ID_LECTURA_CENTRO|ID_LECTURA_RUTA|ID_LECTURA_ITINERARIO|ID_ESTADO|ID_DEPARTAMENTO|ID_MUNICIPIO|CodigoPuntoConsumo|
+-------+-------+-------+-----------+---------+-------+----------+-----------------+---------------+---------------------+---------+---------------+------------+------------------+
|1000012|  EC012|    B20|    1000012|       15|   9005|      2201|             2201|              1|                   10|        3|             26|         151|           1000012|
+-------+-------+-------+-----------+---------+-------+----------+-----------------+---------------+---------------------+---------+---------------+------------+------------------+



In [26]:
join_4 = join_3.join(DIMENSION_GEOGRAFIA,((join_3.ID_ESTADO==DIMENSION_GEOGRAFIA.CodigoDepartamentoGeografia)&\
                                          (join_3.ID_DEPARTAMENTO==DIMENSION_GEOGRAFIA.CodigoMunicipioGeografia)&\
                                          (join_3.ID_MUNICIPIO==DIMENSION_GEOGRAFIA.CodigoLocalidadZonaGeografia)))
join_4.count()

2842627

In [27]:
DIMENSION_GEOGRAFIA.groupBy(col("CodigoDepartamentoGeografia"),\
                            col("CodigoMunicipioGeografia"),col("CodigoLocalidadZonaGeografia")).count()\
.orderBy(col("count").desc()).show()

+---------------------------+------------------------+----------------------------+-----+
|CodigoDepartamentoGeografia|CodigoMunicipioGeografia|CodigoLocalidadZonaGeografia|count|
+---------------------------+------------------------+----------------------------+-----+
|                          1|                       5|                          10|    1|
|                          1|                       6|                        7809|    1|
|                          1|                      12|                        8758|    1|
|                          1|                     542|                        8875|    1|
|                          2|                      24|                         111|    1|
|                          5|                      52|                        9755|    1|
|                          5|                     510|                        7552|    1|
|                          6|                      66|                         605|    1|
|         

In [29]:
DIMENSION_GEOGRAFIA.filter((col("CodigoDepartamentoGeografia")==1)&(col("CodigoMunicipioGeografia")==4)&\
                          (col("CodigoLocalidadZonaGeografia")==9)).show()

+--------------+---------------------------+------------------------+----------------------------+
|LlaveGeografia|CodigoDepartamentoGeografia|CodigoMunicipioGeografia|CodigoLocalidadZonaGeografia|
+--------------+---------------------------+------------------------+----------------------------+
|           159|                          1|                       4|                           9|
+--------------+---------------------------+------------------------+----------------------------+



In [22]:
join_4.filter(col("NIS_RAD")==1000012).show(20)

+-------+-------+-------+-----------+---------+-------+----------+-----------------+---------------+---------------------+---------+---------------+------------+------------------+--------------+---------------------------+------------------------+----------------------------+
|NIS_RAD|EST_SUM|COD_TAR|ID_SERVICIO|ID_BARRIO|ID_CNAE|ID_OFICINA|ID_LECTURA_CENTRO|ID_LECTURA_RUTA|ID_LECTURA_ITINERARIO|ID_ESTADO|ID_DEPARTAMENTO|ID_MUNICIPIO|CodigoPuntoConsumo|LlaveGeografia|CodigoDepartamentoGeografia|CodigoMunicipioGeografia|CodigoLocalidadZonaGeografia|
+-------+-------+-------+-----------+---------+-------+----------+-----------------+---------------+---------------------+---------+---------------+------------+------------------+--------------+---------------------------+------------------------+----------------------------+
|1000012|  EC012|    B20|    1000012|       15|   9005|      2201|             2201|              1|                   10|        3|             26|         151|     

In [30]:
join_4.groupBy(col("NIS_RAD")).count().show()

+-------+-----+
|NIS_RAD|count|
+-------+-----+
|1000012|    1|
|1001925|    1|
|1003076|    1|
|1005174|    1|
|1006909|    1|
|1007975|    1|
|1008678|    1|
|1008941|    1|
|1012072|    1|
|1015454|    1|
|1019956|    1|
|1020674|    1|
|1020879|    1|
|1020964|    1|
|1021122|    1|
|1023669|    1|
|1029458|    1|
|1029703|    1|
|1031165|    1|
|2001826|    1|
+-------+-----+
only showing top 20 rows



In [49]:
DIMENSION_ACTIVIDAD_ECONOMICA = (
    spark.read
    .format("jdbc")
    .option("url", "jdbc:postgresql://postgres/smartrecovery")
    .option("dbtable", "public.\"DimensionActividadEconomica\"")
    .option("user", "admin")
    .option("password", "secret")
    .option("driver", "org.postgresql.Driver")
    .load().alias("DIMENSION_ACTIVIDAD_ECONOMICA")).select("CodigoActividadEconomica")
DIMENSION_ACTIVIDAD_ECONOMICA.printSchema()

root
 |-- CodigoActividadEconomica: integer (nullable = true)



In [50]:
join_5 = join_4.join(DIMENSION_ACTIVIDAD_ECONOMICA,join_3.ID_CNAE ==DIMENSION_ACTIVIDAD_ECONOMICA.CodigoActividadEconomica)
join_5.count()

2842585

In [51]:
join_5_concat = join_5.withColumn("concated_join_col",concat_ws(".",col("ID_LECTURA_CENTRO"),col("ID_LECTURA_RUTA"),col("ID_LECTURA_ITINERARIO")))
join_5_concat.count()


2842585

In [52]:
DIMENSION_ZONA = (
    spark.read
    .format("jdbc")
    .option("url", "jdbc:postgresql://postgres/smartrecovery")
    .option("dbtable", "public.\"DimensionZona\"")
    .option("user", "admin")
    .option("password", "secret")
    .option("driver", "org.postgresql.Driver")
    .load().alias("DIMENSION_ZONA")).select("LlaveZona","CodigoOficina","CodigoItinerario")
DIMENSION_ZONA.printSchema()

root
 |-- LlaveZona: integer (nullable = true)
 |-- CodigoOficina: decimal(10,0) (nullable = true)
 |-- CodigoItinerario: string (nullable = true)



In [53]:
join_6 = join_5_concat.join(DIMENSION_ZONA, ((join_5_concat.ID_OFICINA==DIMENSION_ZONA.CodigoOficina)&\
                                     (join_5_concat.concated_join_col==DIMENSION_ZONA.CodigoItinerario)))
join_6.count()

2842547

In [54]:
final_df = join_6.select(col("NIS_RAD").alias("llavePuntoConsumo"),"CodigoPuntoConsumo","LlaveGeografia",\
                         col("CodigoActividadEconomica").alias("llaveActividadEconomica"),"LlaveZona")

final_df.show()

+-----------------+------------------+--------------+-----------------------+---------+
|llavePuntoConsumo|CodigoPuntoConsumo|LlaveGeografia|llaveActividadEconomica|LlaveZona|
+-----------------+------------------+--------------+-----------------------+---------+
|          1000002|           1000002|          1142|                   9988|     5872|
|          1000003|           1000003|          1142|                   6211|     4047|
|          1000005|           1000005|          1142|                   8014|     6101|
|          1000012|           1000012|          1142|                   9005|     5872|
|          1000014|           1000014|          1142|                   9005|     5872|
|          1000023|           1000023|          1142|                   9005|     5872|
|          1000024|           1000024|          1142|                   9005|     5872|
|          1000046|           1000046|          1142|                   7201|     5872|
|          1000048|           10

In [38]:
join_6.printSchema()

root
 |-- NIS_RAD: decimal(7,0) (nullable = true)
 |-- EST_SUM: string (nullable = true)
 |-- COD_TAR: string (nullable = true)
 |-- ID_SERVICIO: decimal(7,0) (nullable = true)
 |-- ID_BARRIO: decimal(8,0) (nullable = true)
 |-- ID_CNAE: string (nullable = true)
 |-- ID_OFICINA: decimal(4,0) (nullable = true)
 |-- ID_LECTURA_CENTRO: decimal(4,0) (nullable = true)
 |-- ID_LECTURA_RUTA: decimal(4,0) (nullable = true)
 |-- ID_LECTURA_ITINERARIO: decimal(4,0) (nullable = true)
 |-- ID_ESTADO: decimal(9,0) (nullable = true)
 |-- ID_DEPARTAMENTO: decimal(9,0) (nullable = true)
 |-- ID_MUNICIPIO: decimal(9,0) (nullable = true)
 |-- CodigoPuntoConsumo: string (nullable = true)
 |-- LlaveGeografia: integer (nullable = true)
 |-- CodigoDepartamentoGeografia: string (nullable = true)
 |-- CodigoMunicipioGeografia: string (nullable = true)
 |-- CodigoLocalidadZonaGeografia: decimal(10,0) (nullable = true)
 |-- CodigoActividadEconomica: integer (nullable = true)
 |-- concated_join_col: string (null

In [56]:
(
    final_df.write
    .format("jdbc")
    .option("url", "jdbc:postgresql://postgres/smartrecovery")
    .option("dbtable", "public.\"HechoLlaves\"")
    .option("user", "admin")
    .option("password", "secret")
    .option("driver", "org.postgresql.Driver")
    .option("truncate", True) \
    .mode("overwrite")
    .save()
)

In [ ]:
join_6.groupBy(col("NIS_RAD")).count().orderBy(col("count").desc()).show()


In [ ]:
join_6.filter(col("NIS_RAD")==5635811).show(30)

# calculos NORM update

In [25]:
df_ant = spark.read.option("header", "true").parquet(f"s3a://data-lake/preprocessed_tables/calculosNorm.parquet")\
.alias("ant").filter(col("id_mes")==202212).cache()
df_ant.show(1)

+-------+------+----+--------+----------------+-----------+-------------+-------------+--------------------+----------------------+----------------------+---------------+-----------------+-----------------+------------------------+--------------------------+--------------------------+
|nis_rad|id_mes|csmo|  csmo_d|          csmo_n|consumoCero|consumoCero_d|consumoCero_n|acumuladoConsumoCero|acumuladoConsumoCero_d|acumuladoConsumoCero_n|consumoRepetido|consumoRepetido_d|consumoRepetido_n|acumuladoConsumoRepetido|acumuladoConsumoRepetido_d|acumuladoConsumoRepetido_n|
+-------+------+----+--------+----------------+-----------+-------------+-------------+--------------------+----------------------+----------------------+---------------+-----------------+-----------------+------------------------+--------------------------+--------------------------+
|1000305|202212|   8|0.275862|8.39080459770083|          0|            0|            0|                   0|                     0|           

In [30]:
spark.read.option("header", "true").parquet(f"s3a://data-lake/incremental/ConsumosNormFact.parquet")\
.groupBy(col("id_mes")).count().show()

+------+-------+
|id_mes|  count|
+------+-------+
|202301|2309527|
+------+-------+



In [21]:
df_2 = spark.read.option("header", "true").parquet(f"s3a://data-lake/incremental/ConsumosNormFact.parquet")\
.filter(col("id_mes")==202301).select('nis_rad', 'id_mes', 'csmo', 'csmo_d', 'csmo_n').alias("act")
df_2 = df_2.withColumn('consumoCero', when(col('csmo') == 0, 1).otherwise(0))\
    .withColumn('consumoCero_d', when(col('csmo') == 0, 1).otherwise(0))\
    .withColumn('consumoCero_n', when(col('csmo') == 0, 1).otherwise(0))

df_2.show(1)

+-------+------+----+--------+-----------------+-----------+-------------+-------------+
|nis_rad|id_mes|csmo|  csmo_d|           csmo_n|consumoCero|consumoCero_d|consumoCero_n|
+-------+------+----+--------+-----------------+-----------+-------------+-------------+
|1000842|202301|  88|2.750000|83.64583333333333|          0|            0|            0|
+-------+------+----+--------+-----------------+-----------+-------------+-------------+
only showing top 1 row



In [10]:
mySchema = StructType([StructField(c, StringType()) for c in df_ant.columns])

columns_act = ['act.nis_rad',
               'act.id_mes',
               'act.csmo',
               'act.csmo_d',
               'act.csmo_n',
               'act.consumoCero',
               'act.consumoCero_d',
               'act.consumoCero_n',
               'acumuladoConsumoCero',
               'acumuladoConsumoCero_d', 'acumuladoConsumoCero_n', 'consumoRepetido', 'consumoRepetido_d',
               'consumoRepetido_n', 'acumuladoConsumoRepetido', 'acumuladoConsumoRepetido_d',
               'acumuladoConsumoRepetido_n']

In [26]:

df_act = df_2.alias("act")
# acumuladoConsumoCero
join_act_ant = df_ant.join(df_act, col(
    "ant.nis_rad") == col("act.nis_rad"))
join_act_ant = join_act_ant.withColumn("acumuladoConsumoCero", when(col("ant.csmo") != 0, 0).when(col("act.csmo") != 0, 0)
                                       .when(col("act.csmo").isNull(), 0).otherwise(col("ant.acumuladoConsumoCero")+1))
# acumuladoConsumo_d
join_act_ant = join_act_ant.withColumn("acumuladoConsumoCero_d", when(col("ant.csmo_d") != 0, 0).when(col("act.csmo_d") != 0, 0)
                                       .when(col("act.csmo_d").isNull(), 0).otherwise(col("ant.acumuladoConsumoCero_d")+1))
# acumuladoConsumo_n
join_act_ant = join_act_ant.withColumn("acumuladoConsumoCero_n", when(col("ant.csmo_n") != 0, 0).when(col("act.csmo_n") != 0, 0)
                                       .when(col("act.csmo_n").isNull(), 0).otherwise(col("ant.acumuladoConsumoCero_n")+1))
# consumo repetido
join_act_ant = join_act_ant.withColumn("consumoRepetido", when(col("ant.csmo").isNull(), 0).when(col("act.csmo") == 0, 0)
                                       .when(col("act.csmo").isNull(), 0).when(col("act.csmo") != col("ant.csmo"), 0).otherwise(lit(1)))
# consumo repetido_d
join_act_ant = join_act_ant.withColumn("consumoRepetido_d", when(col("ant.csmo_d").isNull(), 0).when(col("act.csmo_d") == 0, 0)
                                       .when(col("act.csmo_d").isNull(), 0).when(col("act.csmo_d") != col("ant.csmo_d"), 0).otherwise(lit(1)))
# consumo repetido_d
join_act_ant = join_act_ant.withColumn("consumoRepetido_n", when(col("ant.csmo_n").isNull(), 0).when(col("act.csmo_n") == 0, 0)
                                       .when(col("act.csmo_n").isNull(), 0).when(col("act.csmo_n") != col("ant.csmo_n"), 0).otherwise(lit(1)))
# ConsumoAcumuladoRepetido
join_act_ant = join_act_ant.withColumn("acumuladoConsumoRepetido", when(col("ant.csmo").isNull(), 0).when(col("act.csmo") == 0, 0)
                                       .when(col("act.csmo").isNull(), 0).when(col("act.csmo") != col("ant.csmo"), 0)
                                       .when(col("ant.csmo") == 0, 2).otherwise(col("ant.acumuladoConsumoRepetido")+1))
# ConsumoAcumuladoRepetido_d
join_act_ant = join_act_ant.withColumn("acumuladoConsumoRepetido_d", when(col("ant.csmo_d").isNull(), 0).when(col("act.csmo_d") == 0, 0)
                                       .when(col("act.csmo_d").isNull(), 0).when(col("act.csmo_d") != col("ant.csmo_d"), 0)
                                       .when(col("ant.csmo_d") == 0, 2).otherwise(col("ant.acumuladoConsumoRepetido_d")+1))
# ConsumoAcumuladoRepetido_n
join_act_ant = join_act_ant.withColumn("acumuladoConsumoRepetido_n", when(col("ant.csmo_n").isNull(), 0).when(col("act.csmo_n") == 0, 0)
                                       .when(col("act.csmo_n").isNull(), 0).when(col("act.csmo_n") != col("ant.csmo_n"), 0)
                                       .when(col("ant.csmo_n") == 0, 2).otherwise(col("ant.acumuladoConsumoRepetido_n")+1))

# df_act_final.show()
df_act_final = join_act_ant.select(columns_act).alias("ant")


In [27]:
df_act_final.show(5)

+-------+------+----+--------+------------------+-----------+-------------+-------------+--------------------+----------------------+----------------------+---------------+-----------------+-----------------+------------------------+--------------------------+--------------------------+
|nis_rad|id_mes|csmo|  csmo_d|            csmo_n|consumoCero|consumoCero_d|consumoCero_n|acumuladoConsumoCero|acumuladoConsumoCero_d|acumuladoConsumoCero_n|consumoRepetido|consumoRepetido_d|consumoRepetido_n|acumuladoConsumoRepetido|acumuladoConsumoRepetido_d|acumuladoConsumoRepetido_n|
+-------+------+----+--------+------------------+-----------+-------------+-------------+--------------------+----------------------+----------------------+---------------+-----------------+-----------------+------------------------+--------------------------+--------------------------+
|1000005|202301| 188|5.875000|178.69791666666667|          0|            0|            0|                   0|                     0|   

In [28]:
df_act_final.count()

1647341